In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
import numpy as np

## Load Pickles of checkpoint charges per byte

In [3]:
from os import getcwd, path, pardir, scandir
import pickle
from math import ceil

In [4]:
run_experiments = True

In [5]:
from os import path, scandir

from experiments.looped_experiment import looped_experiment

class CheckpointEnergy(object):
    config_file = "looped_experiment.json"
    
    def __init__(self, projects_folder=[path.curdir, "Security"], projects=None):
        self.projects_folder = projects_folder
        self.projects = projects
        if self.projects is None:
            self.projects = [f.name for f in scandir(path.join(*self.projects_folder)) if f.is_dir()]
    
    def measure_all(self, *args, **kwargs):
        for project in self.projects:
            self.measure(project, *args, **kwargs)
    
    def measure(self, project, config_file=None, *args, **kwargs):
        if config_file is None:
            config_file = self.config_file
        config_file_path = path.abspath(path.join(*self.projects_folder, project, config_file))
        if not path.isfile(config_file_path):
            raise IOError(f"Config file not found at {config_file_path}")
        looped_experiment(config_file_path, *args, **kwargs)
    
    

In [6]:
a = CheckpointEnergy()

In [7]:
a.projects

['AES', 'AES_Flash', 'TrustZone', 'TrustZone_Flash']

In [9]:
a.measure('AES_Flash', live_plot=True, max_log_time=20, show_lm_plot=2)

Firmware check OK
Chiperase completed successfully

make: Nothing to be done for 'all'.

Firmware check OK
Programming completed successfully.

Starting DGILibExtra with config: 
{'interfaces': [256, 48], 'loggers': [1, 2, 0], 'gpio_delay_time': 0.0007, 'file_name_base': 'experiment_aes_flash', 'plot_pins': [False, False, True, True], 'plot_pins_method': 'line', 'plot_xmax': 5, 'window_title': 'Experiment AES Flash', 'log_folder': 'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Security\\AES_Flash'}



D:\Anaconda3\envs\SAML11-32\lib\site-packages\matplotlib\widgets.py:346: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0, right=0
  ax.set_xlim((valmin, valmax))


DGILibExtra logger_data: Interfaces:
	 256:   power,   samples:   57000
	  48:   gpio,    samples:     805



In [ ]:
a.measure_all(live_plot=False, max_log_time=1000, show_lm_plot=1)

Firmware check OK
Chiperase completed successfully

make: Nothing to be done for 'all'.

Firmware check OK
Programming completed successfully.

Starting DGILibExtra with config: 
{'interfaces': [256, 48], 'loggers': [1, 0], 'gpio_delay_time': 0.0007, 'file_name_base': 'experiment_aes', 'log_folder': 'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Security\\AES'}

DGILibExtra logger_data: Interfaces:
	 256:   power,   samples:   43000
	  48:   gpio,    samples:     405

Firmware check OK
Chiperase completed successfully

make: Nothing to be done for 'all'.

Firmware check OK
Programming completed successfully.

Starting DGILibExtra with config: 
{'interfaces': [256, 48], 'loggers': [1, 0], 'gpio_delay_time': 0.0007, 'file_name_base': 'experiment_aes_flash', 'log_folder': 'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Security\\AES_Flash'}

DGILibExtra logger_data: Interfaces:
	 256:   power,   samples:   52000
	  48:   gpio,    samples:     805

Experiment TrustZone Flash

In [8]:
if run_experiments:
    from Security.TrustZone_Flash.experiment_trustzone_flash import experiment_trustzone_flash
    experiment_trustzone_flash(path.join(getcwd(), "Security", "TrustZone_Flash"))

flash_nsc_store_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone_Flash", "trustzone_flash_store_charge.p"), "rb"))
flash_nsc_load_charge = pickle.load(open(path.join(getcwd(), "Security", "TrustZone_Flash", "trustzone_flash_load_charge.p"), "rb"))

make: Nothing to be done for 'all'.

Firmware check OK
Chiperase completed successfully
Programming completed successfully.

make: Nothing to be done for 'all'.

Firmware check OK
Programming completed successfully.

Start DGILibExtra with config: 
{'interfaces': [256, 48], 'loggers': [1, 0], 'gpio_delay_time': 0.0007, 'file_name_base': 'experiment_trustzone_flash', 'log_folder': 'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Security\\TrustZone_Flash', 'verbose': 0}

DGILibExtra data: Interfaces:
	 256:   power,   samples: 7576000
	  48:   gpio,    samples:   51708

Saved results in: C:\Users\erikw_000\Documents\GitHub\Atmel-SAML11\Experiments\Security\TrustZone_Flash


### Smoothen?

Make lookup function

In [9]:
AES_BLOCK_SIZE = 16
def get_security_charge(security_type, number_of_bytes):
    if security_type is "AES":
        index = ceil(number_of_bytes/AES_BLOCK_SIZE)
        return encrypt_charge[index] + decrypt_charge[index]
    elif security_type is "AES_Flash":
        index = ceil(number_of_bytes/AES_BLOCK_SIZE)
        return aes_encrypt_charge[index] + flash_write_charge[index] + flash_read_charge[index] + aes_decrypt_charge[index]
    elif security_type is "TrustZone":
        return nsc_store_charge[number_of_bytes] + nsc_store_charge[number_of_bytes]
    elif security_type is "TrustZone_Flash":
        return flash_nsc_store_charge[number_of_bytes] + flash_nsc_load_charge[number_of_bytes]
    else:
        return 0 # No security, so 0 C

In [10]:
len(nsc_store_charge)
#16*880

0

In [18]:
security_types = ["None", "AES", "AES_Flash", "TrustZone", "TrustZone_Flash"]
# security_types = ["None", "AES_Flash", "TruseZone_Flash"]
application_names = ["Kalman", "AR", "Dijkstra", "Sorting 1024", "Bit Counting", "FFT 128"]

In [19]:
# app_charge_scale = 1e-4
application_charges = np.array([0.000326812, 1e-10, 1e-10, 0.00001984909516280825944492365031646130546505446545779705047607421875, 0.0000066193645743728758915470737622666064225995796732604503631591796875, 0.00098674225419147416384080262474753908463753759860992431640625]) # TODO
# app_number_of_bytes_scale = 1000
application_number_of_bytes = np.array([4000, 1, 1, 1544 + 2**10, 90, 1975+24*128]) # TODO

In [20]:
base_charge_per_app = dict(zip(application_names, application_charges))
number_of_bytes_per_app = dict(zip(application_names, application_number_of_bytes))

In [21]:
def get_total_charge(application_name, security_type):
    print(application_name, security_type, f"base: {base_charge_per_app[application_name]}, sec: {get_security_charge(security_type, number_of_bytes_per_app[application_name])}")
    return base_charge_per_app[application_name] + get_security_charge(security_type, number_of_bytes_per_app[application_name])

# Bar chart

In [22]:
import numpy as np
import matplotlib.pyplot as plt

In [23]:
# create plot
fig, ax = plt.subplots(figsize=(10,9))
index = np.arange(len(application_names))
bar_width = 0.18
opacity = 0.8

# ax.grid()

colors = ["blue", "red", "green", "cyan", "magenta", "yellow", "black", "white"]

rects = []
for i,security_type in enumerate(security_types):
    rects += ax.bar(index + i*bar_width, [get_total_charge(application_name, security_type) for application_name in application_names], bar_width, label=security_type, color=colors[i], alpha=opacity)
 
ax.set_xlabel('Application')
ax.set_ylabel('Charge [C]')
ax.set_title('Charge per Application')
fig.canvas.set_window_title('Charge per Application')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(application_names)
ax.legend()
 
fig.tight_layout()
fig.show()

Kalman None base: 0.000326812, sec: 0
AR None base: 1e-10, sec: 0
Dijkstra None base: 1e-10, sec: 0
Sorting 1024 None base: 1.984909516280826e-05, sec: 0
Bit Counting None base: 6.619364574372876e-06, sec: 0
FFT 128 None base: 0.0009867422541914742, sec: 0
Kalman AES base: 0.000326812, sec: 0.000266660294291551
AR AES base: 1e-10, sec: 5.187371787026527e-06
Dijkstra AES base: 1e-10, sec: 5.187371787026527e-06
Sorting 1024 AES base: 1.984909516280826e-05, sec: 0.00017271113084657168
Bit Counting AES base: 6.619364574372876e-06, sec: 1.0433995337623398e-05
FFT 128 AES base: 0.0009867422541914742, sec: 0.0003353747052574817
Kalman AES_Flash base: 0.000326812, sec: 0.0003762351464001336
AR AES_Flash base: 1e-10, sec: 8.73951455681015e-06
Dijkstra AES_Flash base: 1e-10, sec: 8.73951455681015e-06
Sorting 1024 AES_Flash base: 1.984909516280826e-05, sec: 0.00024623702644686776
Bit Counting AES_Flash base: 6.619364574372876e-06, sec: 1.5452415137523475e-05
FFT 128 AES_Flash base: 0.000986742254

# Number of Checkpoints

In [24]:
security_types = ["None", "AES_Flash", "TrustZone_Flash"]

In [35]:
min_cap_size_over_sec_charge = 10
number_of_points = 1e5
iterations = 1
min_cap_factor = 10
min_step_size = 1e-7

In [ ]:
opacity = 0.8
colors = ["blue", "red", "green", "cyan", "magenta", "yellow", "black", "white"]

figs = {}
axs = {}
for application_name in application_names:
# for application_name in ["Sorting 1024", "FFT 128"]:
    fig, ax = plt.subplots(figsize=(10,9))
    figs[application_name] = fig
    axs[application_name] = ax
    fig.canvas.set_window_title(application_name)
    for i,security_type in enumerate(security_types):
        security_charge = get_security_charge(security_type, number_of_bytes_per_app[application_name])
        base_charge = base_charge_per_app[application_name]
        capacitor_sizes = np.arange(base_charge/min_cap_factor,base_charge*iterations, (base_charge*iterations-base_charge/min_cap_factor)/number_of_points)
        capacitor_sizes = capacitor_sizes[capacitor_sizes>security_charge]
        y = base_charge/(capacitor_sizes - security_charge)
#         ax.plot(capacitor_sizes, base_charge/(capacitor_sizes - security_charge)/(base_charge/capacitor_sizes), label=security_type, color=colors[i], alpha=opacity)
        ax.plot(capacitor_sizes, y, label=security_type, color=colors[i], alpha=opacity)
        ax.plot(capacitor_sizes, np.floor(y), color=colors[i], alpha=opacity/2)
#         ax.plot(capacitor_sizes, np.floor(base_charge/(capacitor_sizes - security_charge)), '.', label=security_type, color=colors[i], alpha=opacity/2)
#         ax.plot(capacitor_sizes, base_charge/(capacitor_sizes - security_charge), label=security_type, color=colors[i], alpha=opacity)
    ax.plot([base_charge/min_cap_factor, base_charge*iterations], [1,1])
    ax.set_xscale('log')
#     ax.set_yscale('log')
    ax.set_ylim(-1,10)
    ax.set_xlabel('Capacitor Size [C]')
    ax.set_ylabel('Number of Checkpoints')
    ax.set_title(f'Number of Checkpoints per Security Type {application_name}')
    fig.canvas.set_window_title(f"Number of Checkpoints {application_name}")
    ax.legend()
    fig.tight_layout()
    fig.show()

In [27]:
# opacity = 0.8
# colors = ["blue", "red", "green", "cyan", "magenta", "yellow", "black", "white"]

# figs = {}
# axs = {}
# # for application_name in application_names:
# for application_name in ["Sorting 1024", "FFT 128"]:
#     fig, ax = plt.subplots(figsize=(10,9))
#     figs[application_name] = fig
#     axs[application_name] = ax
#     fig.canvas.set_window_title(application_name)
#     security_charge_aes = get_security_charge("AES_Flash", number_of_bytes_per_app[application_name])
#     security_charge_trustzone = get_security_charge("TrustZone_Flash", number_of_bytes_per_app[application_name])
#     base_charge = base_charge_per_app[application_name]
#     capacitor_sizes = np.arange(base_charge/min_cap_factor,base_charge*iterations, (base_charge*iterations-base_charge/min_cap_factor)/number_of_points)
#     capacitor_sizes = capacitor_sizes[capacitor_sizes>security_charge_aes]
# #         ax.plot(capacitor_sizes, base_charge/(capacitor_sizes - security_charge)/(base_charge/capacitor_sizes), label=security_type, color=colors[i], alpha=opacity)
#     y = base_charge/(capacitor_sizes - security_charge_aes)/(base_charge/(capacitor_sizes - security_charge_trustzone))
#     ax.plot(capacitor_sizes, y, label=security_type, color=colors[i], alpha=opacity)
#     ax.plot(capacitor_sizes, np.floor(y), color=colors[i], alpha=opacity/2)
# #         ax.plot(capacitor_sizes, np.floor(base_charge/(capacitor_sizes - security_charge)), '.', label=security_type, color=colors[i], alpha=opacity/2)
# #         ax.plot(capacitor_sizes, base_charge/(capacitor_sizes - security_charge), label=security_type, color=colors[i], alpha=opacity)
#     ax.plot([base_charge/min_cap_factor, base_charge*iterations], [1,1])
#     ax.set_xscale('log')
# #     ax.set_yscale('log')
#     ax.set_ylim(-1,10)
#     ax.set_xlabel('Capacitor Size [C]')
#     ax.set_ylabel('Number of Checkpoints')
#     ax.set_title(f'Number of Checkpoints per Security Type {application_name}')
#     fig.canvas.set_window_title(f"Number of Checkpoints {application_name}")
#     ax.legend()
#     fig.tight_layout()
#     fig.show()

## Scales

In [28]:
print("Cap Size:")
print(f"\tmin: {min(capacitor_sizes)}, max: {max(capacitor_sizes)}")

Cap Size:
	min: 0.00013514717936082669, max: 0.09867323966363496


In [29]:
print("Base Charge:")
for application_name in application_names:
    print(f"\t{application_name}: {base_charge_per_app[application_name]}")

Base Charge:
	Kalman: 0.000326812
	AR: 1e-10
	Dijkstra: 1e-10
	Sorting 1024: 1.984909516280826e-05
	Bit Counting: 6.619364574372876e-06
	FFT 128: 0.0009867422541914742


In [30]:
print("Security Charge:")
for security_type in security_types:
    print(f"\t{security_type} min: {get_security_charge(security_type, min(number_of_bytes_per_app.values()))},  max: {get_security_charge(security_type, max(number_of_bytes_per_app.values()))}")
# number_of_bytes_per_app = dict(zip(application_names, application_number_of_bytes))

Security Charge:
	None min: 0,  max: 0
	AES_Flash min: 8.73951455681015e-06,  max: 0.0004724774345093848
	TrustZone_Flash min: 5.089571227427976e-06,  max: 0.00013435030928496495


In [31]:
base_charge_per_app

{'Kalman': 0.000326812,
 'AR': 1e-10,
 'Dijkstra': 1e-10,
 'Sorting 1024': 1.984909516280826e-05,
 'Bit Counting': 6.619364574372876e-06,
 'FFT 128': 0.0009867422541914742}

In [32]:
min(number_of_bytes_per_app.values())

1

In [33]:
get_security_charge("None", min(number_of_bytes_per_app.values()))

0

In [34]:
get_security_charge("TrustZone_Flash", 4000)

0.00010843217381790772

In [ ]:
1544 + 2**10

In [ ]:
2**13

In [ ]:
0x37b7

In [ ]:
0x1fb7

In [ ]:
0x1fb7+256*24

In [ ]:
620*16

In [ ]:
0x1fb7-256*24

In [ ]:
1975+24*512

In [ ]:
1975+24*128

In [ ]:
print(f"{1975+24*128:x}")

In [ ]:
print(f"{1975+24*0:x}")

## Plot stuff

In [ ]:
# fig = plt.figure(figsize=(10, 6))
# ax = fig.add_subplot(1, 1, 1)
# ax.set_xlabel("Capacitor Size [C]")
# ax.set_ylabel("Number of Checkpoints")

In [ ]:
# capacitor_sizes = range(1,101)
# def get_number_of_checkpoints(application_number):
#     return [application_charges[application_number]/(capacitor_size - get_security_charge("trustzone_flash", application_number_of_bytes[application_number])) for capacitor_size in capacitor_sizes]

In [ ]:
# for application_number in range(5):
#     ax.plot(capacitor_sizes, get_number_of_checkpoints(application_number), label=application_names[application_number])
# ax.legend()
# # ax.set_yscale('log')
# # ax.set_xscale('log')
# # ax.invert_xaxis()

In [ ]:
# fig2 = plt.figure(figsize=(10, 6))
# ax2 = fig2.add_subplot(111, projection='3d')

# ax2.set_xlabel("Capacitor Size [C]")
# ax2.set_ylabel("Number of Bytes")
# ax2.set_zlabel("Number of Checkpoints")

In [ ]:
# capacitor_sizes = range(1,1001)
# application_charge = 50
# number_of_bytes = range(1,1001)

In [ ]:
# def get_number_of_checkpoints(application_charge, capacitor_size, number_of_bytes, security_type="trustzone_flash"):
#     return application_charge/(capacitor_size - get_security_charge(security_type, number_of_bytes))

In [ ]:
# X, Y = np.meshgrid(capacitor_sizes, number_of_bytes)
# Z = np.zeros(shape=(1000,1000))
# for i,capacitor_size in enumerate(capacitor_sizes):
#     for j,number_of_byte in enumerate(number_of_bytes):
#         Z[i,j] = get_number_of_checkpoints(application_charge, capacitor_size, number_of_byte)
# #Z = np.array([np.array([get_number_of_checkpoints(capacitor_size,application_charge) for application_charge in application_charges]) for  capacitor_size in capacitor_sizes])
# Z.shape

In [ ]:
# surf = ax2.plot_surface(X, Y, Z)